In [0]:
%sql SHOW CATALOGS

In [0]:
# Detect if Unity Catalog is enabled
# Robust: works whether the column is 'catalog' or 'catalog_name'
df = spark.sql("SHOW CATALOGS")
first_col = df.columns[0]          # e.g., 'catalog' or 'catalog_name'
catalogs = [row[first_col] for row in df.collect()]
print(catalogs)




if "main" in catalogs:   # Unity Catalog
    USE_CATALOG = "main"
    db_name = "demo"
    full_db_name = f"{USE_CATALOG}.{db_name}"
    print(f"Unity Catalog detected ✅ - using {full_db_name}")
else:                    # Hive metastore only
    USE_CATALOG = None
    db_name = "demo"
    full_db_name = db_name
    print(f"Hive metastore detected ⚠️ - using {full_db_name}")


In [0]:
# Create (or use) a database named 'demo'
spark.sql("CREATE DATABASE IF NOT EXISTS demo")
spark.sql("USE demo")

# Create a tiny DataFrame and save it as a Delta table in Hive metastore
data = [(1, "Alice"), (2, "Bob"), (3, "Carlos")]
df = spark.createDataFrame(data, ["id", "name"])
df.write.mode("overwrite").format("delta").saveAsTable("demo.hello_spark")

# Query it back
display(spark.table("demo.hello_spark"))

# Make an update so you can show Delta history/time travel
spark.sql("UPDATE demo.hello_spark SET name = 'Bobby' WHERE id = 2")
display(spark.sql("DESCRIBE HISTORY demo.hello_spark").select("version","operation","timestamp"))


In [0]:
spark.sql("USE demo")
display(spark.sql("SELECT id, name FROM hello_spark ORDER BY id"))


In [0]:
%sql
USE demo;
SELECT id, name
FROM hello_spark
ORDER BY id;
